In [53]:
import graphlab as gl
gl.canvas.set_target('ipynb')

# Load data, Create word count and tf-idf column 

In [54]:
people = gl.SFrame('people_wiki.gl/')
#people.column_names()

In [55]:
people['word_count'] = gl.text_analytics.count_words(people['text'])

In [56]:
people['tfidf'] = gl.text_analytics.tf_idf(people['word_count'])

TFIDF Across all articles

In [57]:
people_tfidf_table = people[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf'])
people_count_table = people[['word_count']].stack('word_count', new_column_name=['word', 'count']).sort('count', ascending=False)

Rename the columns to append them to the peepz SFrame

In [59]:
people_tfidf_table.rename({'tfidf': 'tfidf_count', 'word': 'tfidf_word'})
people_count_table.rename({'count': 'simple_count', 'word': 'count_word'})

count_word,simple_count
the,442
the,246
in,245
the,210
of,209
of,174
the,173
de,165
and,164
the,151


In [76]:
people_tfidf_table.sort('tfidf_count', ascending=False)
people_count_table.sort('simple_count', ascending=False)

count_word,simple_count
the,442
the,246
in,245
the,210
of,209
of,174
the,173
de,165
and,164
the,151


In [80]:
people_count_table['simple_count'][0]

442L

In [78]:
people_tfidf_table['tfidf_count'][0]

3.8914310119380633

In [51]:
people_stacked = people

In [52]:
people_stacked.add_columns(people_tfidf_table)

RuntimeError: Runtime Exception. Column "tfidf_word" has different size than current columns!

## Good old Elton 

In [12]:
elton = people[people['name'] == 'Elton John']
#elton.head(3)

### Word Count: Stack and order it

In [13]:
elton[['word_count']].stack('word_count', new_column_name = ['word', 'count']).sort('count', ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
he,7
john,7
on,6
since,5


### Same for TFIDF

In [14]:
elton[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf']).sort('tfidf', ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
overallelton,10.9864953892
tonightcandle,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


(Furnish is his husband)

# Measuring Distance 

In [16]:
victoria = people[people['name'] == 'Victoria Beckham']
paul = people[people['name'] == 'Paul McCartney']

In [18]:
print gl.distances.cosine(elton['tfidf'][0], victoria['tfidf'][0])
print gl.distances.cosine(elton['tfidf'][0], paul['tfidf'][0])

0.956700637666
0.825031002922


(Of course) Paul < Victoria

# Building nearest Neighbor Model 

In [83]:
knn_model_count = gl.nearest_neighbors.create(elton, features=['simple_count'], label='name', distance='cosine')
knn_model_tfidf = gl.nearest_neighbors.create(victoria, features=['tfidf'], label='name', distance='cosine') 

RuntimeError: Runtime Exception. Column name simple_count does not exist.

In [21]:
knn_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 0us          |

| Done         |         | 100         | 323.094ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5
